# Neptune Graph Data Loader

In [1]:
# Import necessary libraries
!pip install icecream -q
!pip install langchain-community -q
import pandas as pd
import numpy as np
from neptune_loader.gremlin_csv_generator import generate_gremlin_files

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
graph-notebook 4.6.2 requires numpy<1.24.0, but you have numpy 2.2.5 which is incompatible.
sagemaker 2.241.0 requires numpy<2.0,>=1.9.0, but you have numpy 2.2.5 which is incompatible.


In [2]:
import os
import graph_notebook as gn
config = gn.configuration.get_config.get_config()
host = config.host.split('.')[0]
os.environ["HOST"] = host

In [3]:
print(host)

db-neptune-1


In [2]:
# Source data to the loaded
src_data_location = 'data/data.csv' 
src_format_location = 'neptune_loader/data.json'

In [39]:
# Create gremlin standard files using neptune utility
# Note: It always needs the json file to be created to map the relationship between nodes

generate_gremlin_files(src_data_location,src_format_location)


ic| file_name: 'data/data.csv'
ic| node: {'file_name': 'customer_purchase',
           'label': 'Customer Purchase',
           'node_cols': ['CustomerID:string'],
           'properties': {'Country_of_Residence': 'Country:string',
                          'context_type': 'Customer details',
                          'description': 'Customer Purchase history Tree',
                          'effectiveFrom': '20250401120000',
                          'tree_name': 'Customer Purchase history'}}
ic| cols: ['CustomerID:string']
ic| filter_query: None
ic| nodes_df.columns: Index(['CustomerID:string', '~label', 'Country_of_Residence:string',
                             'context_type:string', 'description:string', 'tree_name:string',
                             'effectiveFrom:string', '~id'],
                            dtype='object')
ic| node: {'file_name': 'product_purchased',
           'label': 'Product Purchased',
           'node_cols': ['InvoiceNo:string', 'StockCode:string'],
    

In [42]:
# Copy the created gremlin files to S3 bucket
prefix_name = "customer-data"

!aws s3 cp nodes/data s3://neptune-data-store-poc/gremlin/{prefix_name}/nodes/ --recursive
!aws s3 cp edges/data s3://neptune-data-store-poc/gremlin/{prefix_name}/edges/ --recursive

upload: nodes/data/data.csv_customer_purchase.csv to s3://neptune-data-store-poc/gremlin/customer-data/nodes/data.csv_customer_purchase.csv
upload: nodes/data/data.csv_product_purchased.csv to s3://neptune-data-store-poc/gremlin/customer-data/nodes/data.csv_product_purchased.csv
upload: edges/data/data.csv_customer_products.csv to s3://neptune-data-store-poc/gremlin/customer-data/edges/data.csv_customer_products.csv


In [26]:
!pip install awscurl

In [54]:
HOST='db-neptune-1.cluster-cvczj9jgzh8p.us-west-2.neptune.amazonaws.com'

!awscurl --service neptune-db --region us-west-2 -X POST \
    -H 'Content-Type: application/json' \
    https://db-neptune-1.cluster-cvczj9jgzh8p.us-west-2.neptune.amazonaws.com:8182/loader -d \
    '{"source" : "s3://neptune-data-store-poc/gremlin/customer-data/edges/","format" : "csv","iamRoleArn" : "arn:aws:iam::274630790798:role/NeptuneLoadFromS3","region" : "us-west-2","failOnError" : "TRUE","parallelism" : "MEDIUM","updateSingleCardinalityProperties" : "FALSE","queueRequest" : "TRUE"}'

{
    "status" : "200 OK",
    "payload" : {
        "loadId" : "b9028ecf-3db3-4ec7-9379-75d8aaa0e4ff"
    }
}


In [59]:
!awscurl --service neptune-db --region us-west-2 -X GET 'https://db-neptune-1.cluster-cvczj9jgzh8p.us-west-2.neptune.amazonaws.com:8182/loader/b9028ecf-3db3-4ec7-9379-75d8aaa0e4ff?details=true&errors=true&page=1&errorsPerPage=3'
 

{
    "status" : "200 OK",
    "payload" : {
        "feedCount" : [
            {
                "LOAD_COMPLETED" : 1
            }
        ],
        "overallStatus" : {
            "fullUri" : "s3://neptune-data-store-poc/gremlin/customer-data/edges/",
            "runNumber" : 1,
            "retryNumber" : 0,
            "status" : "LOAD_COMPLETED",
            "totalTimeSpent" : 147,
            "startTime" : 1746619062,
            "totalRecords" : 531225,
            "totalDuplicates" : 0,
            "parsingErrors" : 0,
            "datatypeMismatchErrors" : 0,
            "insertErrors" : 0
        },
        "errors" : {
            "startIndex" : 0,
            "endIndex" : 0,
            "loadId" : "b9028ecf-3db3-4ec7-9379-75d8aaa0e4ff",
            "errorLogs" : [ ]
        }
    }
}


In [9]:
%%oc
MATCH (n) DETACH DELETE n
WITH collect(n) as cnt
CALL neptune.load({format: 'csv', source: 's3://neptune-data-store-poc/gremlin/customer-data/nodes/',format: 'csv',failOnError: True})
YIELD totalRecords
MATCH (n) 
WHERE n:__Node__ OR n:Chunk OR n.entity OR n.__Entity__
DETACH DELETE n 
WITH totalRecords, count(n) as cnt                
RETURN 'Success'

In [1]:
%%oc
CALL neptune.load({format: 'csv', source: 's3://neptune-data-store-poc/gremlin/customer-data/edges/',format: 'csv',failOnError: True})


In [11]:
!aws s3 cp s3://aws-neptune-customer-samples-us-west-2/workshops/building_genai_apps_using_neptune/sbom_data/ data/

fatal error: An error occurred (404) when calling the HeadObject operation: Key "workshops/building_genai_apps_using_neptune/sbom_data/" does not exist


In [20]:
import boto3
from langchain.embeddings import BedrockEmbeddings
import pandas as pd

df = pd.read_csv("nodes/data/data.csv_product_purchased.csv")
print(df.head(5))
bedrock_client = boto3.client(service_name="bedrock-runtime", region_name="us-west-2")
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0", client=bedrock_client
)
for i in df.index:
    json = df.loc[i].to_json()
    embedding = bedrock_embeddings.embed_query(json)
    df.at[i, "embedding:vector"] = (
        str(embedding).replace(", ", ";").replace("[", "").replace("]", "")
    )
    if i % 25 == 0:
        print(f"Embedding row {i}")
  
df.to_csv("nodes/data/embed_data_csv_product_purchased.csv", index=False)

  InvoiceNo:string StockCode:string             ~label  \
0           536365           85123A  Product Purchased   
1           536365            71053  Product Purchased   
2           536365           84406B  Product Purchased   
3           536365           84029G  Product Purchased   
4           536365           84029E  Product Purchased   

                    description:string context_type:string  \
0   WHITE HANGING HEART T-LIGHT HOLDER     Product details   
1                  WHITE METAL LANTERN     Product details   
2       CREAM CUPID HEARTS COAT HANGER     Product details   
3  KNITTED UNION FLAG HOT WATER BOTTLE     Product details   
4       RED WOOLLY HOTTIE WHITE HEART.     Product details   

   Quantity Purchased:string Invoice Date:string  Unit Price:string  \
0                          6        12/1/10 8:26               2.55   
1                          6        12/1/10 8:26               3.39   
2                          8        12/1/10 8:26               2.

KeyboardInterrupt: 

In [27]:
# !pip install --upgrade --quiet  fastembed 
!pip install -U "fastembed>=0.2.0" -q

ERROR: Cannot install fastembed==0.2.1, fastembed==0.2.2, fastembed==0.2.3, fastembed==0.2.4, fastembed==0.2.5, fastembed==0.2.6, fastembed==0.2.7, fastembed==0.3.0, fastembed==0.3.1, fastembed==0.3.2, fastembed==0.3.3, fastembed==0.3.4, fastembed==0.3.5, fastembed==0.3.6, fastembed==0.4.0, fastembed==0.4.1, fastembed==0.4.2, fastembed==0.5.0, fastembed==0.5.1, fastembed==0.6.0 and fastembed==0.6.1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [29]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

In [30]:
embeddings = FastEmbedEmbeddings()

ImportError: FastEmbedEmbeddings requires `pip install -U "fastembed>=0.2.0"`.

In [5]:
HOST='db-neptune-1.cluster-chow2sm86ya0.us-west-2.neptune.amazonaws.com'

!awscurl --service neptune-db --region us-west-2 -X POST \
    -H 'Content-Type: application/json' \
    https://db-neptune-1.cluster-chow2sm86ya0.us-west-2.neptune.amazonaws.com:8182/loader -d \
    '{"source" : "s3://neptune-strands-store-poc/gremlin/customer-data/nodes/","format" : "csv","iamRoleArn" : "arn:aws:iam::029696018720:role/NeptuneLoadFromS3","region" : "us-west-2","failOnError" : "TRUE","parallelism" : "MEDIUM","updateSingleCardinalityProperties" : "FALSE","queueRequest" : "TRUE"}'

{"detailedMessage":"Unable to connect to s3 endpoint. Provided source = s3://neptune-strands-store-poc/gremlin/customer-data/nodes/ and region = us-west-2. Please verify your S3 configuration.","code":"InvalidParameterException","requestId":"22cb8f35-3413-0988-3971-83950f56a754","message":"Unable to connect to s3 endpoint. Provided source = s3://neptune-strands-store-poc/gremlin/customer-data/nodes/ and region = us-west-2. Please verify your S3 configuration."}
